In [1]:
from requests import post
from app.analysis import Quote
query = 'query UnknownFile($limit_0:[FundLimitOptionInput!]!) {viewer {...Fe}} fragment Fe on Viewer {_screenFunds4s9d1a:screenFunds(filterCategory:["$$"],limit:$limit_0,first:100) {edges {node {symbol}}}}'
variables = {'limit_0':[{'type':'FUND_TOTAL_ASSETS','min':1000000000,'inclusive':True}]}
r = post('https://lens.m1finance.com/graphql', json={'query': query.replace('$$', 'Commodities'), 'variables': variables}).json()
symbols = [n['node']['symbol'] for n in r['data']['viewer']['_screenFunds4s9d1a']['edges']]

In [2]:
quote = Quote(symbols, 120)
f'from {quote.start.date()} to {quote.end.date()} - {len(quote.data.columns)}'

'from 2019-02-28 to 2019-06-28 - 9'

In [3]:
quote.statistics(21)

,21-len,21-mean,21-std,21-shrp,drawdown
Symbols,,,,,
GLD,64,1.434470,3.706687,0.342031,3.986548
IAU,64,1.439845,3.732043,0.341148,3.946330
PHYS,64,1.187525,4.191311,0.243565,5.332083
CEF,64,0.419838,3.931924,0.064389,6.656344
USO,64,-1.567085,9.593393,-0.180724,22.842637
SLV,64,-0.639919,2.920424,-0.276188,7.997266
GSG,64,-1.169534,4.569955,-0.292388,12.269215
PDBC,64,-1.153707,2.753353,-0.479551,9.245172
DBC,64,-1.127469,2.616646,-0.494578,9.057530
